<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>AWS Lambda;</li>
  <li>AWS Step Functions;</li>
  <li>AWS EventBridge.</li>
</ol>

---

# **Exercícios**

## 1\. AWS Lambda

### **1.1. Extração**

Extraindo dados da site da [B3](https://www.b3.com.br/pt_br/) através de uma [API](https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx):

In [ ]:
import json
from datetime import datetime

import requests

# -- setup

URL = 'https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx'

# -- extract

try:
  response = requests.get(URL)
  response.raise_for_status()
except Exception as exc:
  raise exc
else:
  data = json.loads(response.text)
  print(f'1: Json - {data}')

# -- transform

data['taxa'] = data['taxa'].replace(',', '.')
data['indice'] = data['indice'].replace('.', '').replace(',', '.')

data['dataTaxa'] = datetime.strptime(data['dataTaxa'], '%d/%m/%Y').strftime('%Y-%m-%d')
data['dataIndice'] = datetime.strptime(data['dataIndice'], '%d/%m/%Y').strftime('%Y-%m-%d')
data['dataReferencia'] = datetime.now().strftime('%Y-%m-%d')

data_csv = ','.join([v for v in data.values()])

print(f'2 Transformado: - {data}')
print(f'3 csv: - {data_csv}')

1: Json - {'taxa': '11,65', 'dataTaxa': '09/01/2024', 'indice': '42.936,83', 'dataIndice': '10/01/2024'}
2 Transformado: - {'taxa': '11.65', 'dataTaxa': '2024-01-09', 'indice': '42936.83', 'dataIndice': '2024-01-10', 'dataReferencia': '2024-01-09'}
3 csv: - 11.65,2024-01-09,42936.83,2024-01-10,2024-01-09


Dividindo essa etapa em duas: extração e transformação. Logo, temos que:

- Criar um *bucket* no AWS [S3](https://aws.amazon.com/pt/s3/) para salvar o dado original (`bronze`); - OK  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/Criar%20um%20bucket%20no%20AWS%20S3%20para%20salvar%20o%20dado%20original%20bronze.jpg?raw=true)  

- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para extrair o dado original; - OK  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/Criar%20uma%20fun%C3%A7%C3%A3o%20AWS%20Lambda%20para%20extrair%20o%20dado%20original.jpg?raw=true)  

- Criar um *bucket* no AWS [S3](https://aws.amazon.com/pt/s3/) para salvar o dado transformado  (`silver`); - OK  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/-%20Criar%20um%20bucket%20no%20AWS%20S3%20para%20salvar%20o%20dado%20transformado%20silver.jpg?raw=true)  

- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para transformar o dado original; - OK  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/Criar%20uma%20fun%C3%A7%C3%A3o%20AWS%20Lambda%20para%20transformar%20o%20dado%20original.jpg?raw=true)  


- Criar uma função AWS [Lambda](https://aws.amazon.com/pt/lambda/) para criar uma tabela no AWS [Athena](https://aws.amazon.com/pt/athena) apontando para o *bucket* do dado transformado. - OK  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/Criar%20uma%20fun%C3%A7%C3%A3o%20AWS%20Lambda%20para%20criar%20uma%20tabela%20no%20AWS%20Athena%20apontando%20para%20o%20bucket%20do%20dado%20transformado.jpg?raw=true)  





**Abaixo segue os respectivos codigos das Funções Lambda**

- AWS [Lambda](https://aws.amazon.com/pt/lambda/) para *bucket* **bronze**:

In [ ]:
import json
import logging
from datetime import datetime

import boto3
import urllib3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  URL = 'https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx'
  BRONZE_BUCKET = 'modulo38-ebac-genesio-moreira-bronze'

  client = boto3.client('s3')

  date = datetime.now().strftime('%Y-%m-%d')
  filename_json = f'stock-exchange-{date}.json'

  # -- extract

  try:
    http = urllib3.PoolManager()
    response = http.request(url=URL, method='get')
  except Exception as exc:
    raise exc
  else:
    data = json.loads(response.data.decode())
    logging.info(msg=data)

  # -- transform

  ...

  # -- load

  try:
      with open(f'/tmp/{filename_json}', mode='w', encoding='utf8') as fp:
          json.dump(data, fp)
      client.upload_file(Filename=f'/tmp/{filename_json}', Bucket=BRONZE_BUCKET, Key=filename_json)
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

- AWS [Lambda](https://aws.amazon.com/pt/lambda/) para *bucket* **silver**:

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  BRONZE_BUCKET = 'modulo38-ebac-genesio-moreira-bronze'
  SILVER_BUCKET = 'modulo38-ebac-genesio-moreira-silver'

  client = boto3.client('s3')

  date = datetime.now().strftime('%Y-%m-%d')
  filename_csv = f'stock-exchange-{date}.csv'
  filename_json = f'stock-exchange-{date}.json'

  # -- extract

  client.download_file(BRONZE_BUCKET, filename_json, f'/tmp/{filename_json}')

  with open(f"/tmp/{filename_json}", mode='r', encoding='utf8') as fp:
      data = json.load(fp)

  # -- transform

  data['taxa'] = data['taxa'].replace(',', '.')
  data['indice'] = data['indice'].replace('.', '').replace(',', '.')

  data['dataTaxa'] = datetime.strptime(data['dataTaxa'], '%d/%m/%Y').strftime('%Y-%m-%d')
  data['dataIndice'] = datetime.strptime(data['dataIndice'], '%d/%m/%Y').strftime('%Y-%m-%d')

  # -- load

  try:
      with open(f'/tmp/{filename_csv}', mode='w', encoding='utf8') as fp:
          fp.write(','.join([v for v in data.values()]))
      client.upload_file(Filename=f'/tmp/{filename_csv}', Bucket=SILVER_BUCKET, Key=f'data_referencia={date}/{filename_csv}')
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

- AWS [Lambda](https://aws.amazon.com/pt/lambda/) para tabela:

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  SILVER_BUCKET = 'modulo38-ebac-genesio-moreira-silver'

  query = f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS cdi (
    taxa double,
    data_taxa string,
    indice double,
    data_indice string
  )
  PARTITIONED BY (
    data_referencia string
  )
  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
  WITH SERDEPROPERTIES ('separatorChar'=',')
  LOCATION 's3://modulo38-ebac-genesio-moreira-silver/'
  """

  client = boto3.client('athena')

  # -- create

  try:
    client.start_query_execution(
      QueryString=query,
      ResultConfiguration={'OutputLocation': 's3://ebac-genesio-moreira-query-results/'}
    )
  except ClientError as exc:
    raise exc

  # -- update

  try:
    client.start_query_execution(
      QueryString='MSCK REPAIR TABLE cdi',
      ResultConfiguration={'OutputLocation': 's3://ebac-genesio-moreira-query-results'}
    )
  except ClientError as exc:
    raise exc

  return json.dumps(dict(status=True))

- No AWS Athena ficou da seguinte forma:  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/imagem_athena.jpg?raw=true)

## 2\. AWS Step Functions

Replique as atividades do item 2.3 em sua conta.


- Step Functions
Realizado a configuração.  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/print_step_functions1.jpg?raw=true)  



- Apos criado, foi executado, apresentando a seguinte saida:  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/execu%C3%A7%C3%A3o%20do%20step%20function.jpg?raw=true)  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/graph_viewn%20step%20function.jpg?raw=true)  


## 3\. AWS EventBridge

Replique as atividades do item 3.3 em sua conta.

- Foi realizado o EventBridge, a criaçao da regram para execução programada de no perido a cada 24 horas.  

![](https://github.com/Genesio-Moreira87/Modulo38-ebac/blob/main/images/print%20amazon%20eventbridge.jpg?raw=true)  


**Obs** :. o nome ficou modulo28, o padrao seria modulo38. porém appós criado não consigo alterar o nome do EventBridge.